In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=c7f81d456ad69d702980631a7a6163da1ee8fa41f5b5ff23c010fdc649546e0e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 2.6 MB 6.0 MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-py3-none-any.whl size=7053 sha256=fe3f7d848b4b789afffbf13f75d7defab4d8a499f9686fd378e8ccce9ed266e6
  Stored in directory: /root/.cache/pip/wheels/12/40/73/a0765d65e793332b79dfe6c34c713e7c0066ea785191b3f50a
Successfully built language-detector
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3566 sha256=ff106ee784b8053dd60

True

In [4]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [5]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoder
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

In [6]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.5.0', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [38]:
dataset_type = "clinical"
dataset = LDABERT2Dataset(dataset_type=dataset_type,
                           pct_data=0.005,
                          max_seq_length=128,
                           max_segment_length=300,
                           augment_pct=0.005)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_path = './data/lda_bert_2/{}/{}_{}_{}.pkl'.format(dataset_type, 
                                                            dataset.pct_data, 
                                                            dataset.augment_pct,
                                                             dataset.max_segment_length)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors(vectors_path)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors(vectors_path)

something went wrong [Errno 2] No such file or directory: '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/data/lda_bert_2/generated_vectors/clinical/0.005_0.005_300.pkl'
root path /content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2
Preprocessing raw texts ...
sentences length 203
Preprocessing raw texts. Done!
lda sentences length 203
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 203 203 203


In [39]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

In [40]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [41]:
saved_tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([203, 110]), (203, 10))

## Model

In [44]:
model = ContextEncoder(final_dropout=0.5, 
                         dense_neurons=128, 
                         max_sentence_length=128, 
                       bert_trainable=False,
                         gamma=20)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [45]:
checkpoint_filepath = '../models/LDABERT2/simple/clinical-16122-1-pct-1-aug_OX2HS/no-finetune/checkpoint.ckpt'

In [46]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

In [47]:
saved_tokenized_sentences

{'input_ids': <tf.Tensor: shape=(203, 110), dtype=int32, numpy=
array([[  101,  1996,  2691, ...,     0,     0,     0],
       [  101, 10548, 14708, ...,     0,     0,     0],
       [  101,  3056, 16662, ...,     0,     0,     0],
       ...,
       [  101,  2019, 23589, ...,     0,     0,     0],
       [  101,  1037,  6612, ...,     0,     0,     0],
       [  101,  2057,  2031, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(203, 110), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(203, 110), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

## Inference

In [49]:
slice_size = 100

for i in range(0, saved_vectors.shape[0]//slice_size):
    start = i*slice_size
    end = start + slice_size
    print(start, end)

    sliced_tokenized_sentences = {
        'input_ids': saved_tokenized_sentences['input_ids'][start:end],
        'token_type_ids': saved_tokenized_sentences['token_type_ids'][start:end],
        'attention_mask': saved_tokenized_sentences['attention_mask'][start:end]
    }
    sliced_vectors = saved_vectors[start:end]
    
    left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(sliced_tokenized_sentences)
    lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(sliced_vectors)
    
    logits = model([
        left_input, 
        mid_input, 
        right_input, 
        lda_left_input, 
        lda_mid_input, 
        lda_right_input
    ])

    logits_flattened = [float(p) for p in logits]

    predictions = [1 if float(logit) >= 0.5 else 0 for logit in logits_flattened]

    k = 14

    string_predictions = "".join([str(i) for i in predictions])
    string_ground_truth = "".join([str(i) for i in saved_labels[start:end]])
    print(string_predictions)
    print(string_ground_truth)
    overall_windowdiff = windowdiff(string_predictions, string_ground_truth, k)
    overall_pk = pk(string_predictions, string_ground_truth, k)

    print("{},{},{},{}".format(overall_windowdiff, overall_pk, k, k))

0 100
0000000000000000000000000000000000000000000000001000001000000000000100000100000010000100100000000000
1000000000000000000000000000000000000000000000001000000000000000000000000100000000000000000000000000
0.5402298850574713,0.28735632183908044,14,14
100 200
0000000000000100000000000000000000000000000000000000000000000000000000000000000000010000100000000000
0001001000000100000001000000000000001000000010000000000100000001000000010000000000011000000010000000
0.9080459770114943,0.632183908045977,14,14


In [ ]:
[float(p) for p in predictions], saved_labels